# Politics in Vanderbilt

In [1]:
SCHOOL="vanderbilt"
SUBJECT="opinion"
START_YEAR=2015
FINAL_YEAR=2023
DATA_DIR="data"      # should be 'data'
OUTPUT_DIR="output"  # should be 'output'

In [2]:
%pwd

'C:\\Users\\ranar\\ASDRP\\diversity-colleges\\notebooks'

In [3]:
%cd ..

C:\Users\ranar\ASDRP\diversity-colleges


In [4]:
import sys
sys.path.append('src')
import ouraws
import ourgraphs
import textutil

In [5]:
# NOTE: before loading, we need to be in the "diversity-colleges" folder

S3OBJECT_KEY = f"{DATA_DIR}/{SCHOOL}-{SUBJECT}-SNAPSHOT.parquet"
df = ouraws.getFromS3(S3OBJECT_KEY)

In [6]:
df.shape

(474, 6)

In [7]:
df.head(2)

,title,url,body,year,month,day
0,"\n\t\t\t\t\t\t\t\t\t\tBARBIN: Win or lose, Sta...",https://vanderbilthustler.com/2023/03/10/barbi...,"BARBIN: Win or lose, Stackhouse is building a ...",2023,3,10
1,\n\t\t\t\t\t\t\t\t\t\tZHANG: Vanderbilt engine...,https://vanderbilthustler.com/2023/03/06/zhang...,ZHANG: Vanderbilt engineering is failing its s...,2023,3,6


In [8]:
import requests
import os

BIPARTISAN_API_KEY = os.environ.get("BIPARTISAN_API_KEY")

BIPARTISAN_URL = "https://api.thebipartisanpress.com/api/endpoints/beta/robert"

In [9]:
BIPARTISAN_API_KEY

'gAAAAABeVpQJKRM5BqPX91XW2AKfz8pJosk182maAweJcm5ORAkkBFj__d2feG4H5KIeOKFyhUVSY_uGImiaSBCwy2L6nWxx4g=='

In [10]:
df2022 = df[df['year'] == 2022]

articles_list = df2022['body'].to_list()
for article in articles_list:
    payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
    response = requests.post(BIPARTISAN_URL, data=payload)
    print(response.text)
    break

0.970463


In [11]:
df['year'].unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015], dtype=int64)

In [12]:
# SYNCHRONOUS METHOD : TAKES A LONG TIME

results = []

for year in range(2015, 2023):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year

Year: 2015 ... 1
2015	1	-0.219144	-0.219144
Year: 2016 ... 20
2016	20	-40.814286	-2.0407143000000003
Year: 2017 ... 61
2017	61	-192.88907699999996	-3.162116016393442
Year: 2018 ... 63
2018	63	-176.61186099999995	-2.8033628730158724
Year: 2019 ... 64
2019	64	-81.04913400000007	-1.266392718750001
Year: 2020 ... 112
2020	112	-242.85191400000008	-2.1683206607142864
Year: 2021 ... 80
2021	80	-52.135808999999995	-0.6516976124999999
Year: 2022 ... 62
2022	62	-8.333947999999996	-0.1344185161290322


In [13]:
import pandas as pd

RESULTS_FILE = f"{DATA_DIR}/{SCHOOL}-POLARITY.parquet"

results_df = pd.DataFrame.from_records(results)
results_df.to_parquet(RESULTS_FILE)